In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 3


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 08-06 08:51:03] ax.service.ax_client: Generated new trial 48 with parameters {'Drug': 98, 'SL_1': 72, 'SL_2': 0, 'SL_3': 0, 'LL_1': 0, 'LL_2': 0, 'LL_3': 0, 'P_1': 0, 'P_2': 0, 'P_3': 0, 'S_1': 0, 'S_2': 0, 'S_3': 0, 'Water': 0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the rel

19 min


In [6]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,0,0,0,0
1,0,0,0,0
2,1,0,0,0
3,0,0,88,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,11,0,0,0


In [7]:
mask = (unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']] == 0).all(axis=1)

# print the rows that are about to change
print("Rows to be updated (before):")
print(unlabeled_trial.loc[mask])

# set Water to 100 on those rows
unlabeled_trial.loc[mask, 'Water'] = 100

# print the rows after the update
print("\nRows after update:")
print(unlabeled_trial.loc[mask])

Rows to be updated (before):
    trial_index  Drug  SL_1  SL_2  SL_3  LL_1  LL_2  LL_3  P_1  P_2  ...  S_1  \
0            48    98    72     0     0     0     0     0    0    0  ...    0   
1            49    98    72     0     0     0     0     0    0    0  ...    0   
4            52    99    72     0     0     0     0     0    0    0  ...    0   
5            53    99    72     0     0     0     0     0    0    0  ...    0   
6            54    99    72     0     0     0     0     0    0    0  ...    0   
7            55    99    72     0     0     0     0     0    0    0  ...    0   
8            56    99    72     0     0     0     0     0    0    0  ...    0   
10           58    99    72     0     0     0     0     0    0    0  ...    0   
11           59    99    72     0     0     0     0     0    0    0  ...    0   
12           60    92    34     0     0     0     0     0    0    0  ...    0   
13           61    99    72     0     0     0     0     0    0    0  ...    0   

In [8]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,0,0,0,100
1,0,0,0,100
2,1,0,0,0
3,0,0,88,0
4,0,0,0,100
5,0,0,0,100
6,0,0,0,100
7,0,0,0,100
8,0,0,0,100
9,11,0,0,0


In [9]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.0,16.000000,16.000000
mean,55.500000,0.591312,0.408687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125000,0.0,0.125000,0.750000
std,4.760952,0.039520,0.039520,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.341565,0.0,0.341565,0.447214
min,48.000000,0.576000,0.270000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
25%,51.750000,0.579000,0.421000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.750000
50%,55.500000,0.579000,0.421000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.000000
75%,59.250000,0.579000,0.421000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.000000
max,63.000000,0.730000,0.424000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,1.000000,1.000000


In [10]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.0,16.000000,16.00000
mean,55.500000,95.875000,67.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750000,0.0,11.750000,75.00000
std,4.760952,9.985823,12.979471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.744692,0.0,32.181775,44.72136
min,48.000000,59.000000,34.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000
25%,51.750000,98.000000,72.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,75.00000
50%,55.500000,99.000000,72.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,100.00000
75%,59.250000,99.000000,72.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,100.00000
max,63.000000,99.000000,72.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.000000,0.0,100.000000,100.00000


In [11]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.0,16.000000,16.000000,16.0
mean,55.500000,206.959375,143.040625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125.000000,0.0,125.000000,750.000000,0.0
std,4.760952,13.831995,13.831995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,341.565026,0.0,341.565026,447.213595,0.0
min,48.000000,201.600000,94.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
25%,51.750000,202.650000,147.350000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,750.000000,0.0
50%,55.500000,202.650000,147.350000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1000.000000,0.0
75%,59.250000,202.650000,147.350000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1000.000000,0.0
max,63.000000,255.500000,148.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.000000,0.0,1000.000000,1000.000000,0.0


In [12]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug        3.3
SL_1        2.3
SL_2        0.0
SL_3        0.0
LL_1        0.0
LL_2        0.0
LL_3        0.0
P_1         0.0
P_2         0.0
P_3         0.0
S_1         2.0
S_2         0.0
S_3         2.0
Water      12.0
Solvent     0.0
dtype: float64

In [13]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [14]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [15]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [16]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [17]:
updated_script

('protocol/iteration_3_OT_2_protocol_1.py',
 'protocol/iteration_3_OT_2_protocol_2.py')

# Perform experiment

# Results

In [18]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,48,0.576,0.424,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,49,0.576,0.424,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,50,0.576,0.424,0,0,0,0,0,0,0,0,1,0,0,0,2,0
3,51,0.579,0.421,0,0,0,0,0,0,0,0,0,0,1,0,2,0
4,52,0.579,0.421,0,0,0,0,0,0,0,0,0,0,0,1,1,0
5,53,0.579,0.421,0,0,0,0,0,0,0,0,0,0,0,1,1,0
6,54,0.579,0.421,0,0,0,0,0,0,0,0,0,0,0,1,1,0
7,55,0.579,0.421,0,0,0,0,0,0,0,0,0,0,0,1,1,0
8,56,0.579,0.421,0,0,0,0,0,0,0,0,0,0,0,1,1,0
9,57,0.579,0.421,0,0,0,0,0,0,0,0,1,0,0,0,2,0


In [19]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Caution,A1,260.950210,0.165124
1,Caution,A2,268.664034,0.228659
2,Caution,A3,252.139489,0.134430
3,Good,A4,247.153395,0.408490
4,Good,A5,254.141996,0.185321
5,Good,A6,251.078297,0.172687
6,Good,B1,279.336492,0.282058
7,Good,B2,273.098247,0.250667
8,Good,B3,263.515018,0.194251
9,Good,B4,254.555956,0.392846


In [20]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,Formulation 2,250.791230,3.503133,0.255499,0.132644,1
2,Formulation 3,271.983252,7.969452,0.242325,0.044494,1
3,Formulation 4,246.544336,11.405841,0.330669,0.060690,1
4,Formulation 5,255.057438,3.053129,0.179415,0.004521,1
5,Formulation 6,255.599858,9.307955,0.234082,0.035050,1
6,Formulation 7,246.027381,4.847660,0.194133,0.021829,1
7,Formulation 8,250.815282,10.397039,0.232382,0.102890,1
8,Formulation 9,268.858460,4.501796,0.183350,0.021692,1
9,Formulation 10,262.639198,7.608169,0.167788,0.045661,1


In [21]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,48,0.576,0.424,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,49,0.576,0.424,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 2,250.791230,3.503133,0.255499,0.132644,1
2,50,0.576,0.424,0,0,0,0,0,0,0,...,0,0,2,0,Formulation 3,271.983252,7.969452,0.242325,0.044494,1
3,51,0.579,0.421,0,0,0,0,0,0,0,...,1,0,2,0,Formulation 4,246.544336,11.405841,0.330669,0.060690,1
4,52,0.579,0.421,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 5,255.057438,3.053129,0.179415,0.004521,1
5,53,0.579,0.421,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 6,255.599858,9.307955,0.234082,0.035050,1
6,54,0.579,0.421,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 7,246.027381,4.847660,0.194133,0.021829,1
7,55,0.579,0.421,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 8,250.815282,10.397039,0.232382,0.102890,1
8,56,0.579,0.421,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 9,268.858460,4.501796,0.183350,0.021692,1
9,57,0.579,0.421,0,0,0,0,0,0,0,...,0,0,2,0,Formulation 10,262.639198,7.608169,0.167788,0.045661,1


In [22]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,48,0.576,0.424,0,0,0,0,0,0,0,...,1,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
1,49,0.576,0.424,0,0,0,0,0,0,0,...,1,0,Formulation 2,250.791230,3.503133,0.255499,0.132644,1,1152.0,0
2,50,0.576,0.424,0,0,0,0,0,0,0,...,2,0,Formulation 3,271.983252,7.969452,0.242325,0.044494,1,1152.0,0
3,51,0.579,0.421,0,0,0,0,0,0,0,...,2,0,Formulation 4,246.544336,11.405841,0.330669,0.060690,1,1158.0,0
4,52,0.579,0.421,0,0,0,0,0,0,0,...,1,0,Formulation 5,255.057438,3.053129,0.179415,0.004521,1,1158.0,0
5,53,0.579,0.421,0,0,0,0,0,0,0,...,1,0,Formulation 6,255.599858,9.307955,0.234082,0.035050,1,1158.0,0
6,54,0.579,0.421,0,0,0,0,0,0,0,...,1,0,Formulation 7,246.027381,4.847660,0.194133,0.021829,1,1158.0,0
7,55,0.579,0.421,0,0,0,0,0,0,0,...,1,0,Formulation 8,250.815282,10.397039,0.232382,0.102890,1,1158.0,0
8,56,0.579,0.421,0,0,0,0,0,0,0,...,1,0,Formulation 9,268.858460,4.501796,0.183350,0.021692,1,1158.0,0
9,57,0.579,0.421,0,0,0,0,0,0,0,...,2,0,Formulation 10,262.639198,7.608169,0.167788,0.045661,1,1158.0,0


In [23]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
1,1,0.250791,0.003503,0.576,0.0,0.255499,0.132644,0.083333
2,1,0.271983,0.007969,0.576,0.0,0.242325,0.044494,0.166667
3,1,0.246544,0.011406,0.579,0.0,0.330669,0.060690,0.166667
4,1,0.255057,0.003053,0.579,0.0,0.179415,0.004521,0.083333
5,1,0.255600,0.009308,0.579,0.0,0.234082,0.035050,0.083333
6,1,0.246027,0.004848,0.579,0.0,0.194133,0.021829,0.083333
7,1,0.250815,0.010397,0.579,0.0,0.232382,0.102890,0.083333
8,1,0.268858,0.004502,0.579,0.0,0.183350,0.021692,0.083333
9,1,0.262639,0.007608,0.579,0.0,0.167788,0.045661,0.166667


In [24]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
1,1,0.250791,0.003503,0.576,0.0,0.255499,0.132644,0.083333
2,1,0.271983,0.007969,0.576,0.0,0.242325,0.044494,0.166667
3,1,0.246544,0.011406,0.579,0.0,0.330669,0.060690,0.166667
4,1,0.255057,0.003053,0.579,0.0,0.179415,0.004521,0.083333
5,1,0.255600,0.009308,0.579,0.0,0.234082,0.035050,0.083333
6,1,0.246027,0.004848,0.579,0.0,0.194133,0.021829,0.083333
7,1,0.250815,0.010397,0.579,0.0,0.232382,0.102890,0.083333
8,1,0.268858,0.004502,0.579,0.0,0.183350,0.021692,0.083333
9,1,0.262639,0.007608,0.579,0.0,0.167788,0.045661,0.166667


In [25]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [26]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [27]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,48,0.576,0.424,0,0,0,0,0,0,0,...,0.083333,0,Formulation 1,1.000000,0.000000,1.000000,0.000000,0,0.000,0
1,49,0.576,0.424,0,0,0,0,0,0,0,...,0.083333,0,Formulation 2,0.250791,0.003503,0.255499,0.132644,1,0.576,0
2,50,0.576,0.424,0,0,0,0,0,0,0,...,0.166667,0,Formulation 3,0.271983,0.007969,0.242325,0.044494,1,0.576,0
3,51,0.579,0.421,0,0,0,0,0,0,0,...,0.166667,0,Formulation 4,0.246544,0.011406,0.330669,0.060690,1,0.579,0
4,52,0.579,0.421,0,0,0,0,0,0,0,...,0.083333,0,Formulation 5,0.255057,0.003053,0.179415,0.004521,1,0.579,0


In [28]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [29]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [30]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-06 11:04:29] ax.service.ax_client: Completed trial 48 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-06 11:04:29] ax.service.ax_client: Completed trial 49 with data: {'Solu': (0.576, 0), 'Size': (0.250791, 0.003503), 'PDI': (0.255499, 0.132644)}.
[INFO 08-06 11:04:29] ax.service.ax_client: Completed trial 50 with data: {'Solu': (0.576, 0), 'Size': (0.271983, 0.007969), 'PDI': (0.242325, 0.044494)}.
[INFO 08-06 11:04:29] ax.service.ax_client: Completed trial 51 with data: {'Solu': (0.579, 0), 'Size': (0.246544, 0.011406), 'PDI': (0.330669, 0.06069)}.
[INFO 08-06 11:04:29] ax.service.ax_client: Completed trial 52 with data: {'Solu': (0.579, 0), 'Size': (0.255057, 0.003053), 'PDI': (0.179415, 0.004521)}.
[INFO 08-06 11:04:29] ax.service.ax_client: Completed trial 53 with data: {'Solu': (0.579, 0), 'Size': (0.2556, 0.009308), 'PDI': (0.234082, 0.03505)}.
[INFO 08-06 11:04:29] ax.service.ax_client: Completed trial 54 with data: {'Solu': (0.579, 0), '